In [32]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from arch import arch_model 

# --- 1. DATOS REALES (El mercado mexicano) ---
ticker = 'CEMEXCPO.MX'
data = yf.download(ticker, start='2020-01-01', end='2025-01-01')['Close']

# Retornos logarítmicos (multiplicados por 100 para estabilidad numérica en GARCH)
returns = 100 * data.pct_change().dropna()

# --- 2. EL CEREBRO: AJUSTE DEL MODELO GARCH(1,1) ---
# Basado en SFM Cap. 12 (Time Series) y Cap. 14
# Asumimos una media constante y volatilidad GARCH
model = arch_model(returns, vol='Garch', p=1, q=1, dist='t') # dist='t' para Colas Pesadas (Student's t)
res = model.fit(disp='off')

print("\n--- RESUMEN DEL MODELO GARCH ---")
print(res.summary())

# --- 3. FILTERED HISTORICAL SIMULATION (FHS) ---
# Paso A: Obtener los residuos estandarizados (Z)
# Quitamos la volatilidad histórica para dejar solo el "ruido" puro
volatilidad_historica = res.conditional_volatility
std_resid = res.resid / volatilidad_historica

# Paso B: Predecir la volatilidad para MAÑANA
forecasts = res.forecast(horizon=1)
vol_mañana = np.sqrt(forecasts.variance.values[-1, :])[0]

# Paso C: Bootstrap (Simulación)
# Tomamos los residuos pasados al azar y les inyectamos la volatilidad de mañana
n_simulaciones = 10000
simulated_resid = np.random.choice(std_resid, n_simulaciones, replace=True)
simulated_returns = simulated_resid * vol_mañana # Retorno simulado = Ruido Histórico * Volatilidad Actual

# --- 4. CÁLCULO DE VaR y ES (Expected Shortfall) ---
conf_level = 0.95
VaR_95 = np.percentile(simulated_returns, (1-conf_level)*100)
ES_95 = simulated_returns[simulated_returns <= VaR_95].mean() # Promedio de pérdidas en el peor 5%

# Convertir a dinero (Ejemplo: Portafolio de $1,000,000)
capital = 1000000
VaR_cash = (VaR_95 / 100) * capital
ES_cash = (ES_95 / 100) * capital

print(f"\n--- RIESGO ESTIMADO PARA MAÑANA ({ticker}) ---")
print(f"Volatilidad Proyectada: {vol_mañana:.2f}% diaria")
print(f"VaR (95%) con GARCH-FHS: ${abs(VaR_cash):,.2f}")
print(f"Expected Shortfall (CVaR): ${abs(ES_cash):,.2f} <--- La métrica que ven los reguladores")

plt.figure(figsize=(12, 6), dpi = 1500)
plt.style.use('seaborn-v0_8-darkgrid')

# Histograma de retornos simulados
sns.histplot(simulated_returns, bins=100, kde=True, color='#2c3e50', stat='density', alpha=0.6, label='Escenarios Simulados (FHS)')

# Líneas de Riesgo
plt.axvline(VaR_95, color='#e74c3c', linestyle='--', linewidth=2, label=f'VaR 95%: {VaR_95:.2f}%')
plt.axvline(ES_95, color='#f1c40f', linestyle='-', linewidth=2, label=f'Expected Shortfall: {ES_95:.2f}%')

plt.title(f'Pronóstico de Riesgo Dinámico (GARCH-FHS) para {ticker}', fontsize=14)
plt.xlabel('Retorno Diario Simulado (%)')
plt.ylabel('Densidad')
plt.legend()
plt.show()

# Gráfico Extra: Volatilidad en el tiempo (Para mostrar que detectas clusters)
plt.figure(figsize=(12, 4), dpi = 1500)
plt.plot(volatilidad_historica, color='blue', linewidth=0.8, label='Volatilidad Condicional (GARCH)')
plt.title('Volatility Clustering Detectado (2020-2025)', fontsize=12)
plt.legend()
plt.show()

/var/folders/tz/tw3ctq595r744r_sx025n5hc0000gn/T/ipykernel_7381/437040374.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2020-01-01', end='2025-01-01')['Close']
[*********************100%***********************]  1 of 1 completed



--- RESUMEN DEL MODELO GARCH ---
                        Constant Mean - GARCH Model Results                         
Dep. Variable:                  CEMEXCPO.MX   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                            GARCH   Log-Likelihood:               -2904.12
Distribution:      Standardized Student's t   AIC:                           5818.25
Method:                  Maximum Likelihood   BIC:                           5843.93
                                              No. Observations:                 1258
Date:                      Sat, Jan 10 2026   Df Residuals:                     1257
Time:                              21:47:47   Df Model:                            1
                               Mean Model                               
                 coef    std err          t      P>|t|  95.0% Conf. Int.
-------------------------------------------------------

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# ==========================================
# PASO 1: CONSTRUCCIÓN DEL PORTAFOLIO 🏗️
# ==========================================
tickers_solicitados = ['WALMEX.MX', 'CEMEXCPO.MX', 'AMXL.MX', 'GFNORTEO.MX']

# Descarga de datos (5 años)
raw_data = yf.download(tickers_solicitados, start='2020-01-01', end='2025-01-01')['Close']

# Limpieza y Blindaje de Datos
if isinstance(raw_data, pd.Series): 
    raw_data = raw_data.to_frame() # Si solo baja 1, convertir a DataFrame

data = raw_data.dropna(axis=1, how='all') # Eliminar columnas vacías
data = data.ffill().dropna() # Rellenar huecos y limpiar filas NaN

if data.empty:
    raise ValueError("❌ Error Crítico: No hay datos. Revisa tu conexión.")

# Cálculo de Retornos Logarítmicos
individual_returns = np.log(data / data.shift(1)).dropna()

# Ponderación Dinámica (Equiponderado según activos disponibles)
n_activos = len(individual_returns.columns)
weights = np.array([1/n_activos] * n_activos)
print(f"✅ Datos limpios. Activos operativos: {n_activos}")

# Creación del "Activo Sintético" (El Portafolio)
# Multiplicamos retornos individuales por pesos * 100 (para estabilidad numérica del GARCH)
portfolio_returns = individual_returns.dot(weights) * 100 

# ==========================================
# PASO 2: MODELADO GARCH(1,1) (Metodología Härdle) 🧬
# ==========================================
print("⚡ Ajustando modelo GARCH(1,1) con distribución t-Student...")

# Volatilidad GARCH, Media Constante, Dist. t-Student (Colas Pesadas)
model = arch_model(portfolio_returns, vol='Garch', p=1, q=1, mean='Constant', dist='t')
res = model.fit(disp='off')

# Extracción de parámetros clave
volatilidad_condicional = res.conditional_volatility
params_nu = res.params[-1] # Grados de libertad (nu) para la t-Student

# ==========================================
# PASO 3: CÁLCULO DE VaR DINÁMICO 📉
# ==========================================
# Nivel de Confianza: 95%
alpha = 0.05 

# Obtenemos el cuantil exacto de la t-Student ajustada
# Esto es mejor que usar -1.65 (Normal) porque captura el riesgo de cola
q_student = model.distribution.ppf(alpha, params_nu)

# VaR_t = Media + Vol_t * Quantile_t
# Usamos .values para evitar errores de alineación de índices
vec_mean = portfolio_returns.mean()
vec_vol = volatilidad_condicional.values.flatten()
var_series = pd.Series(vec_mean + vec_vol * q_student, index=portfolio_returns.index)

# ==========================================
# PASO 4: VISUALIZACIÓN 1 - BACKTESTING
# ==========================================
# Identificar días donde perdimos más que el VaR (Excepciones)
excepciones = portfolio_returns[portfolio_returns < var_series]

fig, ax = plt.subplots(figsize=(12, 6), dpi = 1500)

# A. Retornos
ax.plot(portfolio_returns.index, portfolio_returns, color='gray', alpha=0.4, label='Retornos Diarios Portafolio', linewidth=1)

# B. Línea de VaR (Nuestra protección)
ax.plot(var_series.index, var_series, color='#c0392b', linestyle='-', linewidth=2, label='VaR 95% Dinámico (GARCH)')

# C. Excepciones (Puntos Magenta)
ax.scatter(excepciones.index, excepciones.values, color='magenta', s=50, zorder=5, edgecolors='black', label='Excepciones (Brechas)')

ax.set_title('Backtesting de Riesgo: GARCH(1,1) en Portafolio MX', fontsize=14, fontweight='bold')
ax.set_ylabel('Retorno Diario (%)')
ax.legend(loc='lower left')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.tight_layout()
plt.savefig('backtesting_result.png')
plt.show()

# ==========================================
# PASO 5: VALIDACIÓN ESTADÍSTICA & DIAGNÓSTICO 🧪
# ==========================================
# A. Reporte Numérico
n_fallos = len(excepciones)
n_total = len(portfolio_returns)
pct_fallos = n_fallos / n_total

print("\n" + "="*40)
print("📊 REPORTE DE VALIDACIÓN (BACKTEST)")
print("="*40)
print(f"Total Observaciones: {n_total}")
print(f"Excepciones Reales:  {n_fallos}")
print(f"Tasa de Fallo:       {pct_fallos:.2%} (Objetivo: 5.00%)")

if 0.04 <= pct_fallos <= 0.06:
    print("STATUS: MODELO ACEPTADO (Zona Verde)")
else:
    print("STATUS: REVISAR MODELO (Fuera de banda)")

# B. Gráficos de Diagnóstico (Residuos)
# Validamos que el modelo haya 'limpiado' la autocorrelación
std_resid = res.resid / volatilidad_condicional
fig_diag = plt.figure(figsize=(10, 8), dpi = 1500)

# QQ-Plot
ax1 = fig_diag.add_subplot(211)
sm.qqplot(std_resid, line='s', ax=ax1, markersize=3)
ax1.set_title('QQ-Plot: Ajuste de Colas (t-Student)', fontsize=12)

# ACF de Residuos Cuadrados
ax2 = fig_diag.add_subplot(212)
plot_acf(std_resid**2, lags=40, ax=ax2, alpha=0.05, title='ACF Residuos² (Validación Heterocedasticidad)')

plt.tight_layout()
plt.show()

/var/folders/tz/tw3ctq595r744r_sx025n5hc0000gn/T/ipykernel_7381/464684836.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_data = yf.download(tickers_solicitados, start='2020-01-01', end='2025-01-01')['Close']
[*********************100%***********************]  4 of 4 completed

1 Failed download:
['AMXL.MX']: YFTzMissingError('possibly delisted; no timezone found')
/var/folders/tz/tw3ctq595r744r_sx025n5hc0000gn/T/ipykernel_7381/464684836.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_nu = res.params[-1] # Grados de libertad (nu) para la t-Student


✅ Datos limpios. Activos operativos: 3
⚡ Ajustando modelo GARCH(1,1) con distribución t-Student...
